# 12: Regime-Stratified Evaluation

**Capstone: Complete Validation Combining All Techniques**

---

## 🚨 If You've Made It This Far: The Final Trap

**What you've learned so far**:
- KFold leaks future data → Use WalkForwardCV
- Gap enforcement prevents target overlap → gap >= horizon
- Persistence is hard to beat on sticky data → Use MASE, MC-SS
- Shuffled target catches feature leakage → Run before trusting results
- Statistical tests need HAC variance → Use dm_test, not ttest

**The final trap**: **Aggregate metrics hide regime-specific failures.**

Consider this scenario:
```
Overall MAE: 0.045
Improvement over persistence: 12%
Shuffled target: PASS
DM test: p = 0.03 (significant!)

Conclusion: Model is great, ship it! ✓
```

**But then you stratify by volatility regime...**

```
LOW volatility:    Improvement = +25%  ← Great!
MEDIUM volatility: Improvement = +15%  ← Good
HIGH volatility:   Improvement = -5%   ← FAILURE!
```

**The model fails exactly when it matters most** — during high volatility (crisis periods, market stress, economic transitions).

The overall metrics looked good because:
- Most data points are LOW/MEDIUM volatility (boring periods)
- HIGH volatility periods are rare but critical
- Averaging hides the failure

**This notebook teaches stratified evaluation** — the final defense against hidden model failures.

---

## What You'll Learn

1. **Why aggregate metrics hide failures** — Regime-specific issues masked by overall performance
2. **Stratified validation pipeline** — `run_gates_stratified()` for regime-aware checks
3. **Complete evaluation template** — Combining DM tests, MC-SS, and conformal prediction

**Prerequisites**: Notebooks 01-11 (all prior notebooks)

---

## The Problem: Aggregate Metrics Hide Regime-Specific Failures

Consider this validation result:

```
Overall MAE: 0.045
Overall improvement over persistence: 12%
Status: PASS
```

**But what if we stratify by regime?**

```
LOW volatility:    MAE = 0.020, improvement = 25%  ← Great!
MEDIUM volatility: MAE = 0.040, improvement = 15%  ← Good
HIGH volatility:   MAE = 0.095, improvement = -5%  ← FAILURE!
```

**The model fails exactly when it matters most** — during high volatility periods.

This notebook teaches you to detect such hidden failures.

In [ ]:
import numpy as np
from sklearn.linear_model import Ridge

from temporalcv.gates import (
    gate_shuffled_target,
    gate_suspicious_improvement,
    run_gates,
    run_gates_stratified,
    GateStatus,
)
from temporalcv.statistical_tests import dm_test, pt_test
from temporalcv.persistence import compute_move_conditional_metrics, compute_move_threshold
from temporalcv.conformal import walk_forward_conformal

np.random.seed(42)
print("temporalcv capstone: regime-stratified evaluation")

In [ ]:
# Generate data with distinct volatility regimes
def generate_ar1(n: int, phi: float, sigma: float) -> np.ndarray:
    y = np.zeros(n)
    y[0] = np.random.normal(0, sigma)
    for t in range(1, n):
        y[t] = phi * y[t - 1] + np.random.normal(0, sigma)
    return y

# LOW volatility regime (periods 0-150)
y_low = generate_ar1(150, phi=0.9, sigma=0.05)

# HIGH volatility regime (periods 150-350)
y_high = generate_ar1(200, phi=0.9, sigma=0.25)
y_high = y_high + y_low[-1]

# MEDIUM volatility regime (periods 350-500)
y_med = generate_ar1(150, phi=0.9, sigma=0.12)
y_med = y_med + y_high[-1]

# Combine
y = np.concatenate([y_low, y_high, y_med])
regime_labels = np.array(["LOW"] * 150 + ["HIGH"] * 200 + ["MEDIUM"] * 150)

# Create features
X = np.column_stack([y[:-2], y[1:-1]])
y_target = y[2:]
regimes = regime_labels[2:]

# Train-test split
train_size = int(len(y_target) * 0.7)
X_train, y_train = X[:train_size], y_target[:train_size]
X_test, y_test = X[train_size:], y_target[train_size:]
regimes_test = regimes[train_size:]

print(f"Train: {len(X_train)}, Test: {len(X_test)}")
print(f"\nTest regime distribution:")
for r in ["LOW", "MEDIUM", "HIGH"]:
    n = np.sum(regimes_test == r)
    print(f"  {r}: {n} ({n/len(regimes_test):.1%})")

In [ ]:
# Train model and get predictions
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Compute overall metrics
overall_mae = np.mean(np.abs(y_test - predictions))
persistence_mae = np.mean(np.abs(np.diff(y_test)))
improvement = 1 - overall_mae / persistence_mae

print(f"Overall MAE: {overall_mae:.4f}")
print(f"Persistence MAE: {persistence_mae:.4f}")
print(f"Overall improvement: {improvement:.1%}")

**Looks good overall!** But let's stratify by regime...

---

## 1. Per-Regime Metrics

First, let's compute metrics stratified by volatility regime:

In [ ]:
print("Per-Regime Performance:")
print("="*60)

for regime in ["LOW", "MEDIUM", "HIGH"]:
    mask = regimes_test == regime
    
    if mask.sum() < 10:
        print(f"{regime}: Insufficient samples (n={mask.sum()})")
        continue
    
    regime_actuals = y_test[mask]
    regime_preds = predictions[mask]
    
    # MAE
    regime_mae = np.mean(np.abs(regime_actuals - regime_preds))
    
    # Persistence baseline (for this regime)
    regime_persistence = np.mean(np.abs(np.diff(regime_actuals)))
    
    # Improvement
    if regime_persistence > 0:
        regime_improvement = 1 - regime_mae / regime_persistence
    else:
        regime_improvement = 0
    
    status = "PASS" if regime_improvement > 0 else "WARN" if regime_improvement > -0.1 else "HALT"
    
    print(f"\n{regime} (n={mask.sum()}):")
    print(f"  MAE: {regime_mae:.4f}")
    print(f"  Persistence MAE: {regime_persistence:.4f}")
    print(f"  Improvement: {regime_improvement:.1%}")
    print(f"  Status: {status}")

**Key insight**: The model may fail in specific regimes while passing overall.

This is why we need **stratified validation**.

---

## 2. Stratified Validation with `run_gates_stratified()`

`run_gates_stratified()` runs validation both:
- **Overall**: All gates on full dataset
- **Per-regime**: Numeric gates on each regime separately

In [ ]:
# First, compute overall gates
overall_gates = [
    gate_shuffled_target(
        model, X, y_target,
        method="effect_size",
        random_state=42
    ),
    gate_suspicious_improvement(
        model_metric=overall_mae,
        baseline_metric=persistence_mae
    )
]

# Run stratified validation
stratified_report = run_gates_stratified(
    overall_gates=overall_gates,
    actuals=y_test,
    predictions=predictions,
    regimes=regimes_test,
    min_n_per_regime=10
)

print(stratified_report.summary())

### Auto-Detect Volatility Regimes

If you don't have regime labels, use `regimes="auto"` to classify based on rolling volatility:

In [ ]:
# Auto-detect regimes from actuals
auto_report = run_gates_stratified(
    overall_gates=overall_gates,
    actuals=y_test,
    predictions=predictions,
    regimes="auto",  # Auto-classify volatility
    volatility_window=13,  # ~1 quarter [T3]
    min_n_per_regime=10
)

print("\nAuto-detected regime counts:")
for regime, count in auto_report.regime_counts.items():
    print(f"  {regime}: {count}")

---

## 3. Per-Regime Statistical Tests

Apply the DM test and MC-SS per regime:

In [ ]:
# Compute move threshold from training data
train_changes = np.diff(y_train)
threshold = compute_move_threshold(train_changes, percentile=70.0)

print("Per-Regime Statistical Tests:")
print("="*60)

for regime in ["LOW", "MEDIUM", "HIGH"]:
    mask = regimes_test == regime
    if mask.sum() < 30:  # DM test needs n >= 30
        print(f"\n{regime}: Insufficient samples for DM test (n={mask.sum()})")
        continue
    
    regime_actuals = y_test[mask]
    regime_preds = predictions[mask]
    persistence = X_test[mask, -1]  # y[t-1]
    
    # DM test: model vs persistence
    model_errors = regime_actuals - regime_preds
    persistence_errors = regime_actuals - persistence
    
    try:
        dm = dm_test(model_errors, persistence_errors, h=1)
        dm_result = f"DM={dm.statistic:.2f}, p={dm.pvalue:.3f}"
    except ValueError as e:
        dm_result = f"Error: {e}"
    
    # MC-SS
    pred_changes = np.diff(regime_preds)
    actual_changes = np.diff(regime_actuals)
    
    mc = compute_move_conditional_metrics(pred_changes, actual_changes, threshold=threshold)
    
    print(f"\n{regime} (n={mask.sum()}):")
    print(f"  {dm_result}")
    print(f"  MC-SS: {mc.skill_score:.3f} (reliable: {mc.is_reliable})")

---

## 4. Per-Regime Conformal Prediction

Check if prediction intervals maintain coverage across regimes:

In [ ]:
# Overall conformal prediction
intervals, quality = walk_forward_conformal(
    predictions, y_test,
    calibration_fraction=0.3,
    alpha=0.10
)

print(f"Overall coverage: {quality['coverage']:.1%} (target: 90%)")
print(f"Mean interval width: {quality['mean_width']:.4f}")

# Per-regime coverage
cal_size = int(len(y_test) * 0.3)
holdout_actuals = y_test[cal_size:]
holdout_regimes = regimes_test[cal_size:]

print("\nPer-Regime Coverage:")
for regime in ["LOW", "MEDIUM", "HIGH"]:
    mask = holdout_regimes == regime
    if mask.sum() < 10:
        continue
    
    regime_coverage = intervals.coverage(holdout_actuals[mask])
    print(f"  {regime}: {regime_coverage:.1%} (n={mask.sum()})")

---

## 5. Complete Evaluation Template

Here's a production-ready function combining everything:

In [ ]:
from typing import Dict, Any, Optional
from dataclasses import dataclass

@dataclass
class CompleteEvaluation:
    """Complete model evaluation results."""
    overall_status: str
    overall_mae: float
    overall_improvement: float
    dm_pvalue: Optional[float]
    mc_ss: float
    mc_reliable: bool
    conformal_coverage: float
    regime_results: Dict[str, Dict[str, Any]]
    worst_regime: str
    
    def summary(self) -> str:
        lines = [
            "="*60,
            "COMPLETE MODEL EVALUATION",
            "="*60,
            "",
            "OVERALL METRICS:",
            f"  Status: {self.overall_status}",
            f"  MAE: {self.overall_mae:.4f}",
            f"  Improvement: {self.overall_improvement:.1%}",
            f"  DM p-value: {self.dm_pvalue:.4f}" if self.dm_pvalue else "  DM: N/A",
            f"  MC-SS: {self.mc_ss:.3f} (reliable: {self.mc_reliable})",
            f"  Conformal coverage: {self.conformal_coverage:.1%}",
            "",
            "PER-REGIME RESULTS:",
        ]
        
        for regime, results in sorted(self.regime_results.items()):
            lines.append(f"  [{regime}] (n={results.get('n', 0)}):")
            lines.append(f"    MAE: {results.get('mae', 0):.4f}")
            lines.append(f"    Improvement: {results.get('improvement', 0):.1%}")
            lines.append(f"    Status: {results.get('status', 'N/A')}")
        
        lines.extend([
            "",
            f"WORST REGIME: {self.worst_regime}",
            "="*60,
        ])
        
        return "\n".join(lines)

def complete_evaluation(
    model,
    X: np.ndarray,
    y: np.ndarray,
    train_end_idx: int,
    regimes: np.ndarray,
    random_state: int = 42,
) -> CompleteEvaluation:
    """
    Run complete model evaluation with stratification.
    
    Combines:
    - Shuffled target gate
    - Suspicious improvement gate
    - DM test
    - MC-SS
    - Conformal prediction
    - Per-regime breakdown
    """
    # Split data
    X_train, y_train = X[:train_end_idx], y[:train_end_idx]
    X_test, y_test = X[train_end_idx:], y[train_end_idx:]
    regimes_test = regimes[train_end_idx:]
    
    # Get predictions
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    persistence = X_test[:, -1]
    
    # Overall metrics
    overall_mae = float(np.mean(np.abs(y_test - predictions)))
    persistence_mae = float(np.mean(np.abs(np.diff(y_test))))
    improvement = 1 - overall_mae / persistence_mae
    
    # Determine overall status
    if improvement > 0.20:
        overall_status = "HALT (suspicious)"
    elif improvement > 0.10:
        overall_status = "WARN"
    elif improvement > 0:
        overall_status = "PASS"
    else:
        overall_status = "FAIL (worse than persistence)"
    
    # DM test
    try:
        model_errors = y_test - predictions
        persistence_errors = y_test - persistence
        dm = dm_test(model_errors, persistence_errors)
        dm_pvalue = dm.pvalue
    except:
        dm_pvalue = None
    
    # MC-SS
    threshold = compute_move_threshold(np.diff(y_train))
    mc = compute_move_conditional_metrics(
        np.diff(predictions), np.diff(y_test), threshold=threshold
    )
    
    # Conformal
    _, quality = walk_forward_conformal(predictions, y_test, alpha=0.10)
    
    # Per-regime results
    regime_results = {}
    worst_regime = None
    worst_improvement = float('inf')
    
    for regime in np.unique(regimes_test):
        mask = regimes_test == regime
        if mask.sum() < 10:
            continue
        
        regime_actuals = y_test[mask]
        regime_preds = predictions[mask]
        
        regime_mae = float(np.mean(np.abs(regime_actuals - regime_preds)))
        regime_pers = float(np.mean(np.abs(np.diff(regime_actuals))))
        regime_imp = 1 - regime_mae / regime_pers if regime_pers > 0 else 0
        
        if regime_imp > 0.20:
            status = "HALT"
        elif regime_imp > 0.10:
            status = "WARN"
        elif regime_imp > 0:
            status = "PASS"
        else:
            status = "FAIL"
        
        regime_results[regime] = {
            "n": int(mask.sum()),
            "mae": regime_mae,
            "improvement": regime_imp,
            "status": status,
        }
        
        if regime_imp < worst_improvement:
            worst_improvement = regime_imp
            worst_regime = regime
    
    return CompleteEvaluation(
        overall_status=overall_status,
        overall_mae=overall_mae,
        overall_improvement=improvement,
        dm_pvalue=dm_pvalue,
        mc_ss=mc.skill_score,
        mc_reliable=mc.is_reliable,
        conformal_coverage=quality['coverage'],
        regime_results=regime_results,
        worst_regime=worst_regime or "N/A",
    )

In [ ]:
# Run complete evaluation
eval_result = complete_evaluation(
    model=Ridge(alpha=1.0),
    X=X,
    y=y_target,
    train_end_idx=train_size,
    regimes=regimes,
)

print(eval_result.summary())

---

## Pitfall Section: Common Mistakes

### Pitfall 1: Regime from Full Series

```python
# WRONG: Compute regime boundaries from all data
all_vol = compute_volatility(y_all)
regime_labels = classify_regimes(all_vol)  # Leaks future!

# RIGHT: Compute from training data only
train_vol = compute_volatility(y_train)
regime_thresholds = np.percentile(train_vol, [33, 67])
regime_labels = classify_with_thresholds(test_vol, regime_thresholds)
```

---

### Pitfall 2: Aggregate-Only Validation

```python
# WRONG: Only check overall metrics
if overall_improvement > 0:
    deploy(model)  # May fail in critical regimes!

# RIGHT: Check per-regime performance
for regime, results in eval_result.regime_results.items():
    if results['status'] == 'FAIL':
        raise ValueError(f"Model fails in {regime} regime")
```

---

### Pitfall 3: Ignoring Masked Regimes

```python
# WRONG: Ignore regimes with few samples
if len(stratified_report.masked_regimes) > 0:
    pass  # Ignore them

# RIGHT: Acknowledge limitations
if len(stratified_report.masked_regimes) > 0:
    print(f"Warning: Could not evaluate: {stratified_report.masked_regimes}")
    print("Consider collecting more data for these regimes")
```

---

## Key Insights

```
★ Insight ─────────────────────────────────────

1. Aggregate metrics hide regime-specific failures
   - Model may pass overall but fail in HIGH volatility
   - Always stratify by regime

2. run_gates_stratified() automates stratification
   - regimes="auto" detects volatility regimes
   - min_n_per_regime masks unreliable regimes

3. Complete evaluation combines all techniques
   - Gates: shuffled_target, suspicious_improvement
   - Tests: DM, PT
   - Metrics: MC-SS, direction accuracy
   - Intervals: conformal coverage

4. Focus on the worst regime
   - Model fails when it matters most
   - HIGH volatility is often most important

───────────────────────────────────────────────
```

---

## Curriculum Summary

Congratulations! You've completed the full temporalcv curriculum.

### Tier 1: Foundation (Why Time-Series is Different)
| Notebook | Key Concept |
|----------|-------------|
| 01 | KFold leakage, WalkForwardCV |
| 02 | Gap enforcement, horizon rule |
| 03 | Persistence baseline, MASE |
| 04 | Autocorrelation, HAC variance |

### Tier 2: Prevention (Detecting and Avoiding Leakage)
| Notebook | Key Concept |
|----------|-------------|
| 05 | Shuffled target gate, permutation testing |
| 06 | Safe feature engineering |
| 07 | Threshold and regime leakage |
| 08 | HALT/WARN/PASS validation workflow |

### Tier 3: Evaluation (High-Persistence and Advanced Metrics)
| Notebook | Key Concept |
|----------|-------------|
| 09 | DM test, PT test, model comparison |
| 10 | MC-SS, move-conditional metrics |
| 11 | Conformal prediction, coverage guarantees |
| **12** | **Regime-stratified evaluation (capstone)** |

---

### What You Can Now Do

1. **Validate correctly**: Use walk-forward CV instead of KFold
2. **Detect leakage**: Run shuffled target tests to catch subtle bugs
3. **Enforce gaps**: Configure proper gaps for multi-step forecasting
4. **Evaluate properly**: Use MC-SS and move-conditional metrics
5. **Quantify uncertainty**: Build prediction intervals with guarantees
6. **Stratify by regime**: Expose hidden failures in critical conditions

---

**Happy forecasting!**